In [1]:
from pyspark.sql.types import *
import json

jschemacsv = '{"fields":[{"metadata":{},"name":"Prscrbr_NPI","nullable":true,"type":"integer"},{"metadata":{},"name":"Prscrbr_Last_Org_Name","nullable":true,"type":"string"},{"metadata":{},"name":"Prscrbr_First_Name","nullable":true,"type":"string"},{"metadata":{},"name":"Prscrbr_City","nullable":true,"type":"string"},{"metadata":{},"name":"Prscrbr_State_Abrvtn","nullable":true,"type":"string"},{"metadata":{},"name":"Prscrbr_State_FIPS","nullable":true,"type":"integer"},{"metadata":{},"name":"Prscrbr_Type","nullable":true,"type":"string"},{"metadata":{},"name":"Prscrbr_Type_Src","nullable":true,"type":"string"},{"metadata":{},"name":"Brnd_Name","nullable":true,"type":"string"},{"metadata":{},"name":"Gnrc_Name","nullable":true,"type":"string"},{"metadata":{},"name":"Tot_Clms","nullable":true,"type":"integer"},{"metadata":{},"name":"Tot_30day_Fills","nullable":true,"type":"integer"},{"metadata":{},"name":"Tot_Day_Suply","nullable":true,"type":"integer"},{"metadata":{},"name":"Tot_Drug_Cst","nullable":true,"type":"float"},{"metadata":{},"name":"Tot_Benes","nullable":true,"type":"integer"},{"metadata":{},"name":"GE65_Sprsn_Flag","nullable":true,"type":"string"},{"metadata":{},"name":"GE65_Tot_Clms","nullable":true,"type":"integer"},{"metadata":{},"name":"GE65_Tot_30day_Fills","nullable":true,"type":"integer"},{"metadata":{},"name":"GE65_Tot_Drug_Cst","nullable":true,"type":"float"},{"metadata":{},"name":"GE65_Tot_Day_Suply","nullable":true,"type":"integer"},{"metadata":{},"name":"GE65_Bene_Sprsn_Flag","nullable":true,"type":"string"},{"metadata":{},"name":"GE65_Tot_Benes","nullable":true,"type":"integer"}],"type":"struct"}'

schemacsv = StructType.fromJson(json.loads(jschemacsv))

StatementMeta(, f423086e-808a-4b6a-ae13-87c4e1daa3ae, 3, Finished, Available)

In [2]:
from pyspark.sql.types import *
import json

jschematable = '{"fields":[{"metadata":{},"name":"Prscrbr_NPI","nullable":true,"type":"integer"},{"metadata":{},"name":"Prscrbr_Last_Org_Name","nullable":true,"type":"string"},{"metadata":{},"name":"Prscrbr_First_Name","nullable":true,"type":"string"},{"metadata":{},"name":"Prscrbr_City","nullable":true,"type":"string"},{"metadata":{},"name":"Prscrbr_State_Abrvtn","nullable":true,"type":"string"},{"metadata":{},"name":"Prscrbr_State_FIPS","nullable":true,"type":"integer"},{"metadata":{},"name":"Prscrbr_Type","nullable":true,"type":"string"},{"metadata":{},"name":"Prscrbr_Type_Src","nullable":true,"type":"string"},{"metadata":{},"name":"Brnd_Name","nullable":true,"type":"string"},{"metadata":{},"name":"Gnrc_Name","nullable":true,"type":"string"},{"metadata":{},"name":"Tot_Clms","nullable":true,"type":"integer"},{"metadata":{},"name":"Tot_30day_Fills","nullable":true,"type":"integer"},{"metadata":{},"name":"Tot_Day_Suply","nullable":true,"type":"integer"},{"metadata":{},"name":"Tot_Drug_Cst","nullable":true,"type":"float"},{"metadata":{},"name":"Tot_Benes","nullable":true,"type":"integer"},{"metadata":{},"name":"GE65_Sprsn_Flag","nullable":true,"type":"string"},{"metadata":{},"name":"GE65_Tot_Clms","nullable":true,"type":"integer"},{"metadata":{},"name":"GE65_Tot_30day_Fills","nullable":true,"type":"integer"},{"metadata":{},"name":"GE65_Tot_Drug_Cst","nullable":true,"type":"float"},{"metadata":{},"name":"GE65_Tot_Day_Suply","nullable":true,"type":"integer"},{"metadata":{},"name":"GE65_Bene_Sprsn_Flag","nullable":true,"type":"string"},{"metadata":{},"name":"GE65_Tot_Benes","nullable":true,"type":"integer"},{"metadata":{},"name":"filename","nullable":true,"type":"string"},{"metadata":{},"name":"year","nullable":true,"type":"string"}],"type":"struct"}'

schematable = StructType.fromJson(json.loads(jschematable))

StatementMeta(, f423086e-808a-4b6a-ae13-87c4e1daa3ae, 4, Finished, Available)

In [3]:
from pyspark.sql.types import *
from  pyspark.sql.functions import *

def loadFullDataFromSource(file_name):
    sqljoin= "SELECT 1 as joinyear1, right(substring_index('" + file_name + "', '.', 1),4) as fourdigityear"
    dfyear = spark.sql(sqljoin)
    df = spark.read.format("csv").option("header","true").schema(schemacsv).load("Files/medicare/demorawfiles/" + file_name)
    df = df.withColumn("filename", input_file_name())
    df = df.withColumn("joinyear2", lit(1))
    df = df.join(dfyear, df.joinyear2 == dfyear.joinyear1, 'inner')
    df = df.drop("joinyear1","joinyear2")
    #display(df) 
    df.write.mode("append").format("delta").option("mergeSchema", "true").option("schema", "schematable").save("Tables/medicarepartd_demo3")
    
full_files = [

    'Medicare_Part_D_Prescribers_by_Provider_and_Drug_Dataset_2013.csv',
    'Medicare_Part_D_Prescribers_by_Provider_and_Drug_Dataset_2014.csv',
    'Medicare_Part_D_Prescribers_by_Provider_and_Drug_Dataset_2015.csv',
    'Medicare_Part_D_Prescribers_by_Provider_and_Drug_Dataset_2016.csv',
    'Medicare_Part_D_Prescribers_by_Provider_and_Drug_Dataset_2017.csv',
    'Medicare_Part_D_Prescribers_by_Provider_and_Drug_2018.csv',
    'Medicare_Part_D_Prescribers_by_Provider_and_Drug_2019.csv',
    'Medicare_Part_D_Prescribers_by_Provider_and_Drug_2020.csv',
    'Medicare_Part_D_Prescribers_by_Provider_and_Drug_2021.csv'
    ]

for file in full_files:
    loadFullDataFromSource(file)

StatementMeta(, f423086e-808a-4b6a-ae13-87c4e1daa3ae, 5, Finished, Available)